In [85]:
import numpy as np
import io

In [139]:
dimension = 200

In [140]:
vocab_length = 186186

In [141]:
def load_pretrained_embeddings(filepath):
# load pretrained embeddings
    count = 0
    word_to_id_lang = {}
    id_to_word_lang = {}
    vectors_lang = []
    with io.open(filepath, 'r', encoding='utf-8', newline='\n', errors='ignore') as f:
        for i, line in enumerate(f):
            if i == 0:
                header = line.split()
                assert len(header) == 2
                assert dimension == int(header[1])
            else:
                if i%20000 == 0 :
                        print("Reading %s file %0.2f percent done"%(filepath,float(i)/20000.0))
                word, vector_rep = line.rstrip().split(' ', 1)
                word = word.lower()
                vector_rep = np.fromstring(vector_rep, sep=' ')
                if vector_rep.shape[0] == dimension :
                    if word not in word_to_id_lang :
                        word_to_id_lang[word] = count
                        vectors_lang.append(vector_rep[None])
                        count+=1
                    if count == vocab_length :
                        break

                else:
                    continue
    id_to_word_lang = {value: key for key, value in word_to_id_lang.items()}
    embeddings = np.concatenate(vectors_lang, 0)
    return word_to_id_lang,id_to_word_lang,embeddings

In [142]:
w2i,i2w,embedding=load_pretrained_embeddings("./simi_sememe.embedding")

Reading ./simi_sememe.embedding file 1.00 percent done
Reading ./simi_sememe.embedding file 2.00 percent done
Reading ./simi_sememe.embedding file 3.00 percent done
Reading ./simi_sememe.embedding file 4.00 percent done
Reading ./simi_sememe.embedding file 5.00 percent done
Reading ./simi_sememe.embedding file 6.00 percent done
Reading ./simi_sememe.embedding file 7.00 percent done
Reading ./simi_sememe.embedding file 8.00 percent done
Reading ./simi_sememe.embedding file 9.00 percent done


In [199]:
i2w[0]

'w-年'

In [187]:
from sklearn.metrics.pairwise import cosine_similarity

In [205]:
comp_sim = cosine_similarity(embedding[w2i["w-年"]].reshape(1,-1),embedding)

In [208]:
similar_indices = comp_sim.argsort().flatten()[-10:] 

In [209]:
for i in similar_indices:
    print(i2w[i])

w-改称
w-元年
d-ゴーイン・バック・トゥ・ニューオーリンズ
w-同年
w-月
w-昭和
w-なり年
w-後年
w-翌年
w-年


In [149]:
doc_graph_dic = {}

for line in open('../graph_building/doc.graph'):
    source = line.split()[0].split("-")[1]
    taget = line.split()[1].split("-")[1]
    raletion = line.split()[2]
    doc_graph_dic[source+"-"+taget]=raletion

In [150]:
list(doc_graph_dic.items())[1]

('月-ロデリック・アリアス', '1.2651257248936925')

In [151]:
topic_graph_dic = {}

for line in open('../graph_building/topic.graph'):
    source = "".join(line.split()[0:-2]).split("-")[1]
    taget = line.split()[-2].split("-")[1]
    raletion = line.split()[-1]
    topic_graph_dic[source+"|||"+taget]=raletion

In [181]:
list(topic_graph_dic.items())[1]

('ロデリック・アリアス-2', '0.1184195660024464')

In [185]:

def topic_to_word(topic):
    words_embedding =[]
    doc_list = []
    for key, value in topic_graph_dic.items():
        if topic == key.split("-")[1] and float(value) > 0.05:
            doc_list.append(key.split("-")[0])
    for key, value in doc_graph_dic.items():
        if key.split("-")[1] in doc_list and key.split("-")[0] !="":
            word=key.split("-")[0]
            
            words_embedding.append(embedding[w2i["w-"+word]])
    comp_sim = cosine_similarity(embedding[w2i["t-"+topic]].reshape(1,-1),words_embedding)
    similar_indices = comp_sim.argsort().flatten()[-10:]
    for i in similar_indices:
        print(i2w[i])
                

In [186]:
topic_to_word("6")

KeyError: 370996

In [184]:
w2i['w-日']

3